# Práctica de aprendizaje semi supervisado

En esta práctica vamos a implementar el algoritmo `self-training` para abordar un problema de aprendizaje semi-supervisado con la metodología inductive learning. 

En concreto, para desarrollar la práctica vamos a trabajar con el dataset Ionosphere. Este dataset contiene la información de un problema en el que un sistema de radar recoge información de 16 antenas con el objetivo de ver si los electrones presentan algún tipo de estructura en la ionosfera (Bueno) o no (Malo). Toda la información de este dataset se puede encontrar en la URL: https://archive.ics.uci.edu/ml/datasets/Ionosphere.

Para leer los datos del dataset se facilita la clase dataset (`dataset.py`). Dicha clase tiene un método llamado `lecturaDatos` con los siguientes parámetros de entrada y de salida:

* Entrada: nombre del archivo que contiene los datos del problema a resolver
* Salida: un objeto dataset que sigue la estructura de los datasets nativos de scikit-learn.

El objeto dataset tiene los siguientes campos con diferente información:
* data: contiene los datos leídos del fichero.
* target: contiene las clases del problema.
* feature_names: es una lista con los nombres de las variables de entrada del problema.
* target_names: es una lista con los nombres de las clases del problema.

Para utilizarla, en primer lugar hay que importar el paquete dataset (además importaremos todos las clases necesarias para el desarrollo de la práctica).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, model_selection, metrics
import dataset as ds
from test_helper import Test

Realiza la lectura de los datos almacenados en el fichero `ionosphere.dat` y guardar el resultado en una variable llamada ion.

In [2]:
# Lectura de los datos del problema y guardado del resultado en la variable llamada ion
ion = ds.lecturaDatos("ionosphere.dat")

Paso 1: Una vez leído el dataset vamos a simular un problema de aprendizaje semi-supervisado. En primer lugar vamos a dividir el dataset en dos conjuntos (entrenamiento y test) mediante el método de validación hold-out. Utilizar para ello la función train_test_split de la librería model_selection, cuya información la podéis encontrar en la URL: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split

Utilizar el 30% de los datos para test y el valor 2 como semilla para realizar el particionado.

In [3]:
# Realizamos el particionamiento hold-out
X_train_ini, X_test_ini, y_train_ini, y_test_ini = model_selection.train_test_split(ion.data, ion.target, test_size=0.3, random_state=2)

En esta práctica vamos a utilizar como clasificador una máquina de soporte vectorial (SVM). La librería que contiene las máquinas de soporte vectorial se llama `svm` (la hemos importado al principio de la práctica) y la clase correspondiente al clasificador que vamos a utilizar se llama `SVC`. Toda la información de dicha clase se encuentra en la URL: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

Vamos a obtener su porcentaje de acierto en circunstancias normales (aprendizaje supervisado).

Paso 2: Crear una SVM (utilizar los valores por defecto en la llamada al constructor salvo el parámetro `probability` que debéis asignarlo a `True`y el `random_state` que debe tomar el valor 1), aprender el clasificador utilizando los datos de entrenamiento y predecir las clases de los datos de test. Calcular y mostrar el porcentaje de acierto sobre 100%.

In [4]:
# Llamamos al constructor de SVC (parámetro probability a True y la semilla a 1)
cl = svm.SVC(probability=True,random_state=1)
# Entrenamiento del clasificador con los ejemplos de entrenamiento
cl = cl.fit(X_train_ini, y_train_ini)
# Prediccion de las clases de los ejemplos de test
pr = cl.predict(X_test_ini)
# Obtenemos e imprimimos el porcentaje de acierto
accuracyAS = metrics.accuracy_score(y_test_ini, pr)*100
print ("Con hold-out original 100%, resultado en test: " + str(accuracyAS))

Test.assertEquals(round(accuracyAS, 2), 85.85, 'Accuracy en aprendizaje supervisado incorrecto')

Con hold-out original 100%, resultado en test: 85.8490566038
1 test passed.


Paso 3: Una vez obtenidos ambos conjuntos de ejemplos vamos a simular el problema de aprendizaje semi-supervisado.  Para ello vamos a volver a aplicar la función que nos particiona un conjunto de datos en entrenamiento y test (`train_test_split`) pero esta vez en lugar de pasar como argumento de entrada todo el dataset solamente utilizaremos los datos de entrenamiento obtenidos en la etapa anterior. En esta ocasión los datos de la salida de la función train_test_split (X_train, X_test, y_train, y_test) correspondientes a los datos de entrenamiento serán el conjunto L y las clases de dicho conjunto. Los datos de test serán el conjunto U y sus clases asociadas serán ignoradas. Utilizar un 70% de los datos para test (corresponde a la situación de tener un 30% de los datos disponibles, en L, para entrenar). Utilizar 2 como valor de semilla.

In [5]:
# Realizamos el particionamiento hold-out usando como entrada los ejemplos de entrenamiento obtenidos anteriormente
L, U, L_target, a = model_selection.train_test_split(X_train_ini, y_train_ini, test_size=0.7, random_state=2)

Paso 4: entrenar la SVM con los datos y las clases asociadas a L y predecir las clases de los datos de test (los obtenidos en el paso 1). Calcular el porcentaje de acierto.

In [6]:
# Llamamos al constructor de SVC (parámetro probability a True y la semilla a 1)
cl = svm.SVC(probability=True,random_state=1)
# Entrenamiento del clasificador con los ejemplos del subconjunto L
cl = cl.fit(L, L_target)
# Prediccion de las clases de los ejemplos de test iniciales
pr = cl.predict(X_test_ini)
# Obtenemos e imprimimos el porcentaje de acierto sin aplicar aprendizaje semi-supervisado
accuracySinSSL = metrics.accuracy_score(y_test_ini, pr)*100
print "Con " + str((1-0.7)*100) + "% de train sin SSL, resultado en test: " + str(accuracySinSSL)

Test.assertEquals(round(accuracySinSSL, 2), 77.36, 'Accuracy en aprendizaje supervisado incorrecto')

Con 30.0% de train sin SSL, resultado en test: 77.358490566
1 test passed.


Paso 5: aplicar el algoritmo self-training para etiquetar y aprender un clasificador con los datos de L y con los datos de U siguiendo estas 11 etapas
1.	Entrenar la SVM con los ejemplos y las clases de L
2.	Predecir las probabilidades de las clases de los datos en U (no se debe usar `predict` directamente sino que se debe usar el método `predict_proba`). Este método devuelve para cada ejemplo la probabilidad de cada clase (la clase con más probabilidad sería la predicha). Por tanto, devuelve una lista de listas (tantas filas como ejemplos y tantas columnas como clases).
3.	Ordenar dichas probabilidades por cada ejemplo (por filas, eje 1). Utilizar `np.sort` y ordenando de menor a mayor (por defecto). Guardar el resultado en una variable auxiliar.
4.	Ordenar de nuevo las probabilidades de cada ejemplo pero esta vez para coger los índices (`np.argsort`). Guardar el resultado en otra variable auxiliar. La clase predicha estará en la última columna ya que la mayor predicción obtenida en el paso anterior también estará en dicha columna.
5.	Ordenar las probabilidades de la última columna obtenidas en el paso 3 (las de las predicciones) para coger los índices.
6.	Seleccionar los ejemplos de U correspondientes a los k últimos índices obtenidos en el paso 5.Serán los ejemplos con mayores confianzas en su predicción. K es el número de ejemplos a añadir en cada iteración del algoritmo.
7.	Eliminar los ejemplos anteriores de U. Utilizar la función `np.delete(matriz, indicesEliminar, 0)`, el 0 es por eliminar por filas (eje 0).
8.	Seleccionar las clases predichas para estos ejemplos. Serán las guardadas en las últimas k posiciones del resultado de la etapa 4.
9.	Añadir los ejemplos obtenidos en la etapa 6 a L. Utilizar la función `np.vstack((L, ejemplosAñadir))`.
10.	Añadir las clases predichas para dichos ejemplos a las clases asociadas a L. Utilizar la función `np.hstack((clasesL,clasesPredichasEjemplosAñadidos))`.
11.	Repetir este proceso hasta que no haya más ejemplos en U.

Utilizar 1 como valor de k.

In [7]:
# Llamamos al constructor de SVC (parámetro probability a True y la semilla a 1)
cl = svm.SVC(probability=True,random_state=1)
# Asignamos el valor de k (número de ejemplos a etiquetar en cada iteración)
k = 1
# Mientras haya ejemplos en U
while U.shape[0]>0:
    # Entrenamiento del clasificador con los ejemplos del subconjunto L    
    cl = cl.fit(L, L_target)
    # Prediccion de las probabilidades de los ejemplos del subconjunto U (no etiquetados)
    pr = cl.predict_proba(U)
    # Ordenar las probabilidades por filas
    prAux = np.sort(pr,axis=1)
    # Obtener los índices (clases) al ordenar las probabilidades por filas
    clasesAux = np.argsort(pr,axis=1)
    # Obtener los índices al ordenar la última columna de las probabilidades ordenadas anteriormente
    indices = np.argsort(prAux[:,-1])
    # Seleccionar los k últimos ejemplos de U
    ejemplos = U[indices[-k:]]
    # Seleccionar los k últimas clases predichas para los ejemplos de U
    clases = np.argmax(clasesAux[indices[-k:]], axis=1)
    # Añadir al subconjunto L los ejemplos etiquetados en la iteración
    L = np.vstack((L,ejemplos))
    # Añadir las clases de los ejemplos etiquetados en la iteración (al subconjunto L_target)
    L_target = np.hstack((L_target,clases))
    # Eliminar de U los ejemplos etiquetados en la iteración
    U =  np.delete(U, indices[-k:], 0)

Test.assertEquals(map(lambda x: int(x), L_target[:10]), [1, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'Clases etiquetadas incorrectamente')

1 test passed.


Paso 6: Una vez obtenido el conjunto agrandado en el paso 5 (utilizando k=1), entrenar la SVM con él y predecir las clases de los ejemplos de test (los obtenidos en el paso 1). Calcular el porcentaje de acierto. ¿Mejora con respecto al porcentaje de acierto obtenido en el paso 4?

In [8]:
# Llamamos al constructor de SVC (parámetro probability a True y la semilla a 1)
cl = svm.SVC(probability=True,random_state=1)
# Entrenamiento del clasificador con los ejemplos del subconjunto L
cl = cl.fit(L, L_target)
# Prediccion de las clases de los ejemplos de test iniciales
predicciones3 = cl.predict(X_test_ini)
# Obtenemos e imprimimos el porcentaje de acierto tras aplicar aprendizaje semi-supervisado
accuracyConSSL = metrics.accuracy_score(y_test_ini, predicciones3)*100
print "Con " + str((1-0.7)*100) + "% de train con SSL, resultado en test: " + str(accuracyConSSL)

Test.assertEquals(round(accuracyConSSL, 2), 80.19, 'Valores de accuracy incorrectos')

Con 30.0% de train con SSL, resultado en test: 80.1886792453
1 test passed.


Paso 7: Realizar un bucle para determinar el porcentaje de ejemplos de test utilizados para formar los conjuntos L y U (en el paso 3 lo asignábamos al 70%). En este bucle se deben utilizar todos los valores entre el 5% y el 95% con saltos del 5%. En cada iteración realizar los pasos del 3 al 6. En cada iteración almacenar el resultado obtenido al utilizar aprendizaje semi supervisado y al no utilizarlo. Finalmente mostrar ambos vectores en una gráfica comparativa.

In [ ]:
# Llamamos al constructor de SVC (parámetro probability a True y la semilla a 1)
cl = svm.SVC(probability=True,random_state=1)
# Asignamos el valor de k (número de ejemplos a etiquetar en cada iteración)
k = 1
# Listas para almacenar el rendimiento con y sin aplicar aprendizaje supervisado para cada porcentaje de ejemplos
sinSSL = list([])
conSSL = list([])
# Por cada porcentaje de ejemplos a utilizar como test
for porcentaje in np.arange(0,0.96,0.05):
    # Realizamos el particionamiento hold-out usando como entrada los ejemplos de entrenamiento obtenidos en el paso 1
    L, U, L_target, a = model_selection.train_test_split(X_train_ini, y_train_ini, test_size=porcentaje, random_state=2)
    # Entrenamiento del clasificador con los ejemplos del subconjunto L 
    cl = cl.fit(L, L_target)
    # Rendimiento SIN aplicar aprendizaje semi-supervisado
    # Prediccion de las clases de los ejemplos de test iniciales
    pr = cl.predict(X_test_ini)
    # Obtenemos e imprimimos el porcentaje de acierto sin aplicar aprendizaje semi-supervisado
    accuracySinSSL = metrics.accuracy_score(y_test_ini, pr)*100
    print "Con " + str((1-porcentaje)*100.0) + "% de train sin SSL, resultado en test: " + str(accuracySinSSL)
    
    # Aplicamos el aprendizaje semi-supervisado
    # Mientras haya ejemplos en U
    while U.shape[0]>0:
        # Entrenamiento del clasificador con los ejemplos del subconjunto L 
        cl = cl.fit(L, L_target)
        # Prediccion de las probabilidades de los ejemplos del subconjunto U (no etiquetados)
        pr = cl.predict_proba(U)
        # Ordenar las probabilidades por filas
        prAux = np.sort(pr,axis=1)
        # Obtener los índices (clases) al ordenar las probabilidades por filas
        clasesAux =np.argsort(pr,axis=1)
        # Obtener los índices al ordenar la última columna de las probabilidades ordenadas anteriormente
        indices = np.argsort(prAux[:,-1])
        # Seleccionar los k últimos ejemplos de U 
        ejemplos = U[indices[-k:]]
        # Seleccionar los k últimas clases predichas para los ejemplos de U 
        clases = np.argmax(clasesAux[indices[-k:]], axis=1)
        # Añadir al subconjunto L los ejemplos etiquetados en la iteración
        L = np.vstack((L,ejemplos))
        # Añadir las clases de los ejemplos etiquetados en la iteración (al subconjunto L_target)
        L_target = np.hstack((L_target,clases))
        # Eliminar de U los ejemplos etiquetados en la iteración
        U = np.delete(U, indices[-k:], 0)
        
    # Entrenamiento del clasificador con los ejemplos del subconjunto L
    cl = cl.fit(L, L_target)
    # Prediccion de las clases de los ejemplos de test iniciales
    pr = cl.predict(X_test_ini)
    # Obtenemos e imprimimos el porcentaje de acierto tras aplicar aprendizaje semi-supervisado
    accuracyConSSL = metrics.accuracy_score(y_test_ini, pr)*100
    print "Con " + str((1-porcentaje)*100.0) + "% de train con SSL, resultado en test: " + str(accuracyConSSL)
    # Añadimos los rendimientos a la lista correspondiente
    sinSSL.append(accuracySinSSL)
    conSSL.append(accuracyConSSL)

# Creamos la figura
plt.figure
plt.hold(True)
plt.xticks(1-np.arange(0,0.96,0.05), size = 'small', color = 'b', rotation = 45)
plt.xlabel("Porcentaje de train")
plt.ylabel("Accuracy obtenido")
plt.title("Comparativa entre SSL y no SSL")
plt.plot(1-np.arange(0,0.96,0.05),sinSSL,'r', linewidth = 2, label = 'Sin SSL')
plt.plot(1-np.arange(0,0.96,0.05),conSSL,'g', linewidth = 2, label = 'Con SSL')
plt.legend(loc = 4)
plt.show()

Test.assertEquals(map(lambda x: round(x, 2), sinSSL), [85.85, 84.91, 84.91, 84.91, 82.08, 79.25, 81.13, 80.19, 81.13, 80.19, 82.08, 80.19, 79.25, 78.30, 77.36, 71.70, 73.58, 71.70, 66.98, 57.55], 'Valores de accuracy sin SSL incorrectos')
Test.assertEquals(map(lambda x: round(x, 2), conSSL), [85.85, 84.91, 84.91, 83.02, 83.02, 83.02, 83.02, 83.02, 83.96, 83.96, 83.96, 83.96, 83.96, 84.91, 80.19, 79.25, 80.19, 71.7, 78.3, 57.55], 'Valores de accuracy con SSL incorrectos')

Con 100.0% de train sin SSL, resultado en test: 85.8490566038
Con 100.0% de train con SSL, resultado en test: 85.8490566038
Con 95.0% de train sin SSL, resultado en test: 84.9056603774
Con 95.0% de train con SSL, resultado en test: 84.9056603774
Con 90.0% de train sin SSL, resultado en test: 84.9056603774
Con 90.0% de train con SSL, resultado en test: 84.9056603774
Con 85.0% de train sin SSL, resultado en test: 84.9056603774
Con 85.0% de train con SSL, resultado en test: 83.0188679245
Con 80.0% de train sin SSL, resultado en test: 82.0754716981
Con 80.0% de train con SSL, resultado en test: 83.0188679245
Con 75.0% de train sin SSL, resultado en test: 79.2452830189
Con 75.0% de train con SSL, resultado en test: 83.0188679245
Con 70.0% de train sin SSL, resultado en test: 81.1320754717
Con 70.0% de train con SSL, resultado en test: 83.0188679245
Con 65.0% de train sin SSL, resultado en test: 80.1886792453
Con 65.0% de train con SSL, resultado en test: 83.0188679245
Con 60.0% de train sin